In [17]:
import requests, pickle, json, zipfile, dict2xml, xmltodict, os, shutil, re
from pathlib import Path
from datetime import date, timedelta, datetime
import pandas as pd

today=date.today().isoformat()
year=2025
month = (date.today().replace(day=1) - timedelta(days=1)).month#month=7-----------------------------<=Check month!
code_mo="660017"
baseurl="https://ecp.mis66.ru"
api_person = baseurl + "/api/Person"
api_person_list_url = baseurl + "/api/PersonList"
api_police = baseurl + "/api/Polis"
api_usluga = baseurl + "/api/EvnUsluga"
api_usluga_par = baseurl + "/api/EvnUslugaPar"
api_usluga_list = baseurl + "/api/EvnUslugaList"
api_visit = baseurl + "/api/EvnVizitPL"
ecp_searchData = baseurl + "/?c=Search&m=searchData"
ecp_tap_url=baseurl + '/?c=EvnPL&m=loadEvnPLEditForm'
ecp_visit_url=baseurl + '/?c=EvnVizit&m=loadEvnVizitPLEditForm'
ecp_delete_usluga_url=baseurl + '/?c=EvnUsluga&m=deleteEvnUsluga'
ecp_load_usluga_url=baseurl + '/?c=EvnUslugaPar&m=loadEvnUslugaParEditForm'
ecp_person_url=baseurl + '/?c=Person&m=getPersonEditWindow'
cookie_file=Path("cookies.txt")
os.chdir('C:/Users/User/Anaconda')
today=date.today()
pcr_gepatity_ready=False
pcr_covid_ready=False
fibros_ready=False
cts_ready=False

def yesterday_range(date=today):
    yesterday = today - timedelta(days=1)
    yesterday_string = yesterday.strftime("%d.%m.%Y")
    return yesterday_string+" - " + yesterday_string

def month_range(year=datetime.now().year,month=datetime.now().month):
    
    range_start=date(year,month,1).strftime("%d.%m.%Y")
    range_end=(date(year,month+1,1)-timedelta(days=1)).strftime("%d.%m.%Y")
    return range_start+" - " + range_end
    
def load_cookies():
    print("Loading cookies")
    if cookie_file.is_file():
        print("Cookies found!")
        with open(cookie_file, 'rb') as f:
            session.cookies.update(pickle.load(f))
    else:
        print("Cookies not found")

def save_cookies():
    print("Saving cookies")
    with open(cookie_file, 'wb') as f:
       pickle.dump(session.cookies, f)
    
def login():
    print("Logining")
    headers = {
            "Login": "GIBBot1",
            "Password": "Qwerty1!"
    }
    response=session.request("GET", f"{baseurl}/api/user/login", headers=headers).json()
    save_cookies
    print(response)
    
def sex_is_magic(patronymic):
    if type(patronymic) != str:
        return 2
    if len(patronymic)<4:
        return 2
    if (patronymic[-3:]=='вич' or patronymic[-4:]=='оглы' or patronymic[-4:]=='угли'):
        return 1 
    return 2

def ecp_find_tap_by_num(num):
    payload = {
        'PersonPeriodicType_id' : '1', 
        'SearchFormType' : 'EvnPL', 
        'EvnPL_NumCard' : num, 
        'Vizit_Date_Range' : date_range,
        'Date_Type' : '1', 
        'AddressStateType_id' : '1', 
        'PersonCardStateType_id' : '1', 
        'EvnPS_disDate_Range' : date_range}
    return session.request("GET", ecp_searchData, params=payload).json()

def search_vizit(tap_id):
    payload = {
        'EvnPLBase_id' : tap_id
        }
    return session.request("GET", api_visit, params=payload).json()

def delete_usluga(usluga_id):
    payload = {
        'class' : 'EvnUslugaCommon',
        'id' : usluga_id
        }
    return session.request("POST", ecp_delete_usluga_url, params=payload).json()

def search_usluga(tap_id):
    return session.request("GET", api_usluga_list, params={'Evn_pid' : tap_id, }).json()

def get_usluga(usluge_id):
    return session.request("GET", api_usluga, params={'EvnUsluga_id' : usluge_id, }).json()

def ecp_load_usluga(uslugapar_id):
    return session.request("POST", ecp_load_usluga_url, params={'EvnUslugaPar_id' : uslugapar_id}).json()

def ecp_get_person(person_id):
    return session.request("POST", ecp_person_url, params={'person_id' : person_id}).json()

def ecp_search_person(name='', patronymic = '', surname='', birthday='', police='', snils=''):
    if snils!='':
        if type(snils)==str:
            list_snils = re.findall(r'\d+', snils)
            snils = ''.join(list_snils)
        if type(snils)==int:
            snils="{:011d}".format(snils)
        snils=snils[:3]+"-"+snils[3:6]+"-"+snils[6:9]+"-"+snils[9:]
    params={
        "PersonFirName_FirName": name,
        "PersonSecName_SecName": patronymic,
        "PersonSurName_SurName": surname,
        "PersonBirthDay_BirthDay": birthday,
        "Polis_Num": police,
        "Person_Snils": snils,
    }
    return session.request("POST", baseurl + '/?c=Person&m=getPersonSearchGrid', params=params).json()

def ecp_get_direction(year):
    return session.request("POST", baseurl + '/?c=EvnDirection&m=getEvnDirectionNumber', 
                           params={'year' : year}).json()

def parse_snils(snils):
    if snils == '' or snils is None or snils == 0:
        return ''
    if type(snils)==str:
        list_snils = re.findall(r'\d+', snils)
        snils = ''.join(list_snils)
    if type(snils)==int:
        snils="{:011d}".format(snils)
    return snils[:3]+"-"+snils[3:6]+"-"+snils[6:9]+" "+snils[9:]

#ct_april ct_list=[{'date' : '2024-03-12', 'police' : '6647240842000620','usluga_id':'200880','usluga':'A06.09.005', 'ds' : '5775'}]
# ct_list=[]
date_range=month_range(year,month)

session = requests.session()
load_cookies()
login()

Loading cookies
Cookies found!
Logining
{'error_code': 0, 'sess_id': '6p1i7b566t35gbi71fd4sp0ct4'}


In [2]:
import pandas as pd
from openpyxl import load_workbook
import os
def snils_to_prikreplenie(snils):
    ecp_search_person_resp=ecp_search_person(snils=snils)
    if len(ecp_search_person_resp['data'])==0:
        return 'no pacients found ' + str(len(ecp_search_person_resp['data']))
    if len(ecp_search_person_resp['data'])>1:
        return 'many pacients found ' + str(len(ecp_search_person_resp['data']))
    return ecp_search_person_resp['data'][0]['Lpu_Nick']
    # return ecp_search_person_resp['data'][0]['Person_Snils']
        #       str(ecp_search_person_resp['data'][0]['Polis_Num']),
        #       str(ecp_search_person_resp['data'][0]['Person_IsDead']),
        #       ecp_search_person_resp['data'][0]['PersonSurName_SurName'],
        #       ecp_search_person_resp['data'][0]['PersonFirName_FirName'],
        #       ecp_search_person_resp['data'][0]['PersonSecName_SecName'],
        #       str(ecp_search_person_resp['data'][0]['PersonBirthDay_BirthDay']),
        #       ecp_search_person_resp['data'][0]['Lpu_Nick'],
        #       sep=";",
        # )
def process_excel_file(file_path, process_func):
    """
    Process Excel file to add 'Прикрепление' column based on 'СНИЛС' values.
    
    Args:
        file_path: Path to the Excel file
        process_func: Function that converts SNILS to Prikreplenie value
    """
    # First, let's find the header row and the СНИЛС column
    wb = load_workbook(file_path)
    ws = wb.active
    
    header_row = None
    snils_col = None
    
    # Search for the header row containing 'СНИЛС'
    for row_idx, row in enumerate(ws.iter_rows(values_only=True), 1):
        if 'СНИЛС' in row:
            header_row = row_idx
            snils_col = row.index('СНИЛС') + 1  # Convert to 1-based index for openpyxl
            break
    
    if header_row is None or snils_col is None:
        raise ValueError("Could not find 'СНИЛС' column in the Excel file")
    
    # Find the last column to add our new column
    last_col = ws.max_column + 1
    
    # Add the header for the new column
    ws.cell(row=header_row, column=last_col, value='Прикрепление')
    
    # Process each row below the header
    for row_idx in range(header_row + 1, ws.max_row + 1):
        snils_value = ws.cell(row=row_idx, column=snils_col).value
        
        # Skip empty cells
        if snils_value:
            print('Apply the transformation function to snils', snils_value,end='. ')
            
            prikreplenie_value = process_func(str(snils_value))
            
            print('Write the result to the new column:',prikreplenie_value)
            ws.cell(row=row_idx, column=last_col, value=prikreplenie_value)
    
    # Create a temporary file name
    temp_file = file_path + ".temp"
    
    # Save to temporary file
    wb.save(temp_file)
    wb.close()
    
    # Replace original file with the new one
    os.replace(temp_file, file_path)
    
    print(f"Successfully processed file: {file_path}")

process_excel_file(r"C:\Users\User\Anaconda\2025-09\ГИБ HCV ПЦР сентябрь всё.xlsx", snils_to_prikreplenie)

ValueError: Could not find 'СНИЛС' column in the Excel file

In [8]:
asulon_snils = ['11966934300', '11417430515', '10942779072', '02591335141', '08567502194', '11879210070', '09676837039', '02171758230', '11205618099', '02850691458', '09980053709', '05831915572', '11389593992', '01092010075', '12298615675', '11212690300', '04923718876', '02591518351', '11454891459', '07274435066', '01711207999', '11733564044', '03775415371', '02749626782', '10933709252', '02711434311', '02598557396', '14800624842', '07406735969', '18594486941', '11681120223', '05505485756', '13657536888', '13108184520', '11812733132', '05328758677', '13743035140', '01685882991', '02781429865', '06984352109', '01848871890', '04054358125', '01310343772', '02454610727', '11896503991', '12496470477', '20058959258', '13427519149', '13448097671', '13482215948', '13920645861', '11048353215', '04725025942', '11817173549', '02593167661', '10889747414', '12949271794', '02863090958', '08107885980', '14803165952', '02811844034', '13612546034', '10899065899', '05947622600', '07218047344', '12203641597', '12539249868', '04294185966', '13935604777', '12921452746', '17012149822', '11430769228', '10291045205', '12481514743', '06491819087', '12545865169', '07965832730', '13649605380', '09808485319', '01745544247', '10962070139', '11231394505', '05833178574', '13434638350', '17110440510', '07242878678', '08563949519', '01744596564', '12949625500', '07894725634', '02595766491', '02074901621', '08005647644', '01037659926', '02340302279', '06254229548', '04910210617', '11402873922', '14172577155', '01410202155', '06579919732', '13793070374', '12476813368', '13426446749', '08586396430', '15140836234', '01302747093', '00889553695', '11286208032', '05832602152', '05539614981', '08511080538', '02722112805', '12797509201', '10718329847', '12802101804', '02738047856', '11426103602', '06754859111', '14625429261', '12043423799', '01663030918', '07895117414', '07711341142', '02774826379', '13686433588', '02074961538', '02712150000', '05562695788', '06253880163', '13289363681', '04564539271', '02830569247', '01662693254', '02337501920', '02728311841', '07745513486', '10128639119', '12391203826', '02338725947', '07850100550', '13975708914', '02230170881', '14875031478', '13657536888', '13441475238', '02667173059', '11342812212', '14587372499', '12017320993', '03451280220', '02074508619', '09032388657', '13108184520', '05510221602', '14157107030', '04067874467', '07982792943', '06491214255', '11812733132', '12711532219', '01310733482', '02576886999', '13638950588', '13215185116', '05509844980', '06161632335', '02540451108', '13915202745', '08219101236', '05328758677', '02728809268', '11489686804', '02832374242', '18821710480', '13743035140', '06826910993', '08586515109', '07661639594', '15135404224', '07858925234', '04117511711', '02171658125', '02335028304', '01685882991', '11943254453', '05757492807', '03867166892', '02337571739', '02781429865', '06984352109', '02590569262', '02292835751', '15231255119', '08108534350', '01848871890', '01703227201', '02868966921', '02171509815', '04054358125', '12464082339', '01517960441', '01310343772', '02454610727', '07092061948', '13667273080', '07514917473', '15151871651', '02732240719', '11896503991', '12496470477', '10505647018', '13902117935', '15814562880', '20058959258', '02307420902', '11941096350', '11542543730', '13610329723', '13427519149', '05506698172', '06491754085', '02776046969', '15835725899', '02486850876', '01604592121', '02787186794', '17710698894', '01308553312', '07185473987', '13448097671', '17007784362', '19140631248', '04844247570', '13482215948', '12920872662', '12545876679', '02595736482', '11626093033', '02570830740', '03360756641', '13920645861', '02789943818', '11048353215', '02190929440', '04725025942', '11817173549', '04915135149', '02593167661', '10989090293', '10889747414', '12949271794', '13799499141', '02863090958', '08107885980', '14803165952', '02811844034', '13612546034', '10899065899', '05947622600', '10826904050', '11800955235', '02218732723', '10982832882', '02951479270', '07218047344', '12203641597', '13231801104', '01850782554', '12539249868', '04294185966', '15629516183', '13935604777', '12921452746', '14770902374', '11941560551', '01746521442', '00675873774', '17012149822', '07078235670', '11279744166', '11430769228', '15480024846', '10291045205', '14286035556', '12481514743', '07029193958', '06491819087', '12545865169', '07946594228', '12203291697', '07965832730', '17053271545', '13649605380', '09808485319', '05992376515', '01745544247', '10962070139', '02788970715', '11231394505', '08452472880', '05833178574', '14802950158', '13434638350', '17110440510', '13921942771', '07242878678', '08563949519', '01744596564', '01093151906', '06922630974', '02183140001', '12949625500', '07894725634', '02595766491', '02074901621', '08005647644', '01037659926', '02340302279', '14240153003', '06254229548', '05576710380', '07407231340', '04910210617', '11402873922', '13302955327', '08164011435', '12651684057', '14172577155', '02381656242', '02711667940', '01410202155', '02950024628', '17710660566', '06579919732', '13793070374', '12476813368', '02593087158', '13426446749', '08586396430', '11633783857', '15140836234', '02635348747', '01302747093', '13985853323', '05509698187', '00889553695', '13282028531', '07966436929', '11286208032', '13506173836', '05832602152', '05539614981', '08511080538', '02728087256', '02722112805', '01035112870', '12797509201', '10718329847', '14757507189', '12802101804', '02738047856', '02827724767', '08551786905', '11426103602', '02744295861', '06754859111', '01648006631', '12994912609', '14625429261', '12043423799', '02254812826', '01663030918', '07895117414', '01518631326', '12667462479', '01302098688', '07711341142', '09289807326', '02774826379', '06945356805', '13686433588', '01590215118', '11065800312', '01309838333', '06905837191', '07906771503', '11817869788', '13909847401', '06606600144', '13714623649', '14209765762', '08110361614', '09734471602', '04068513238', '05510805325', '12632758760', '07858940937', '11307836025', '12775861395', '17326060247', '06706988405', '07975919243', '05284503148', '03572917063', '12022125673', '01308139000', '13046551528', '19078394917', '13593237370', '18850965425', '11952900863', '12607962362', '03951763073', '04632621538', '14519146761', '02207249810', '08676592431', '12286820761', '00803430803', '02834912153', '11265071317', '03775490789', '11949862607', '14305543024', '20569354674', '14465051147', '02190772332', '13840198059', '12194765263', '15355911368', '15964267915', '12630664236', '02453353627', '02775317363', '11358711848', '11226030082', '05092275143', '12539328965', '12583775081', '13799044908', '13107729632', '05863959317', '11966934300', '01933919161', '11417430515', '12775919703', '02781185560', '04419684369', '15008964656', '09432701158', '11976204067', '07804045760', '12429684166', '15138406546', '12003967714', '02646961370', '10873654068', '02453673645', '11891714373', '10942779072', '11305779433', '02591335141', '08567502194', '11879210070', '02171761623', '13547623870', '11955464177', '02776833487', '03867149791', '12861537367', '11460436926', '07462873189', '18076645696', '02074005694', '02181644322', '06162134425', '09676837039', '02711327714', '02171758230', '11209746431', '11205618099', '02850691458', '09980053709', '02274865253', '12270144812', '05831915572', '11389593992', '01092010075', '12298615675', '02950052532', '11212690300', '02171769538', '02951597781', '04923718876', '06807899115', '02591518351', '15723126450', '11406590021', '11454891459', '07274435066', '01711207999', '11733564044', '03775415371', '02749626782', '04025450908', '02850775262', '10400738191', '10933709252', '02711434311', '02598557396', '04892367903', '01915858468', '14800624842', '05511300401', '06746085791', '07406735969', '20049547133', '18594486941', '13895954227', '13527363555', '01961970068', '07029257756', '05508924872', '02485481664', '02648871076', '11681120223', '11051846613', '12300302264', '05505485756', '02291477948', '02643738352', '11747280864', '09937789056', '20013278993', '01589298288', '10796269489', '01300278276', '13352606029', '08505372666', '06918123678', '09677249128']
snils_list=['00261120204', '00803365212', '00804142399', '00806129013', '00808829354', '01032109260', '01067124703', '01157677644', '01162245091', '01245485020', '01308248206', '01310196987', '01310441772', '01310580281', '01315677160', '01324037488', '01344928939', '01346747339', '01382344624', '01382527834', '01384738251', '01433122992', '01447082447', '01486471759', '01487642563', '01552516926', '01588346975', '01649274257', '01649626565', '01653951047', '01693051035', '01733447839', '01733566443', '01745728661', '01847654069', '01847654069', '01849334768', '01849334768', '01849465985', '01850576349', '01855897091', '01910103188', '01910393425', '02001765582', '02001829178', '02004158677', '02004158677', '02004172671', '02071520695', '02074005694', '02075495136', '02092854842', '02126878033', '02171240494', '02171748025', '02171767837', '02172896449', '02174647135', '02177268142', '02178341434', '02212512275', '02253765939', '02255058318', '02255405515', '02255687753', '02259466555', '02268151026', '02268171032', '02270344719', '02274881453', '02308584836', '02316856637', '02336197433', '02337453228', '02339318636', '02339695363', '02340457207', '02372403919', '02375581251', '02376194047', '02392410322', '02520441194', '02520441194', '02523557329', '02540446519', '02540919233', '02572038027', '02576694687', '02576694687', '02578015146', '02579491082', '02579562382', '02582484157', '02592810958', '02593048350', '02593125241', '02593483671', '02595100437', '02600491305', '02605421826', '02610925318', '02613412001', '02623598534', '02624347431', '02629156752', '02635342230', '02643791760', '02644246437', '02644716044', '02645458558', '02645631445', '02645748466', '02645912352', '02648497886', '02651194536', '02651862753', '02653964266', '02655866074', '02656012731', '02656166859', '02657132039', '02657363967', '02661122935', '02664507350', '02664773876', '02667724373', '02667738687', '02675469480', '02711523714', '02711707623', '02711952131', '02718734863', '02722720929', '02724182028', '02727488572', '02729075961', '02735908059', '02736412439', '02736412439', '02736932171', '02737652969', '02738604458', '02739439878', '02746760872', '02748727985', '02749351466', '02749569291', '02776600666', '02779440887', '02781767582', '02801509317', '02806440635', '02811957077', '02811964650', '02827769688', '02831293742', '02831538039', '02833485862', '02850381847', '02868922093', '02951407548', '03288018659', '03354229430', '03354473643', '03357954775', '03363112410', '03641251221', '04005562503', '04032434396', '04059240228', '04124990134', '04133294418', '04260407011', '04284338454', '04329390048', '04371380843', '04459385585', '04547183362', '04615969581', '04632641643', '04725195260', '04885358911', '05010113418', '05051479534', '05051503909', '05265673873', '05283065347', '05506313631', '05506315938', '05507454452', '05507548360', '05522113312', '05639470384', '05757479815', '05809877814', '05810180433', '05810550945', '05815422477', '05831915572', '05937138892', '05983880828', '05983900808', '06187484287', '06220718222', '06220776135', '06253219136', '06253924965', '06254880774', '06441672457', '06624172858', '06714827579', '06759783735', '06807903713', '06841874497', '06859065913', '06889962854', '06952181989', '06984739436', '07029238045', '07031249117', '07031280111', '07076645982', '07092957756', '07095992202', '07118623448', '07122958657', '07199594015', '07210313804', '07250282136', '07255107547', '07269141270', '07271135066', '07291626676', '07309582069', '07328437572', '07372825682', '07406746166', '07407207242', '07407207242', '07417926987', '07458952916', '07459546405', '07478243697', '07483778515', '07502993473', '07553424368', '07573091783', '07597423814', '07610856971', '07661467793', '07661668298', '07681048886', '07744304876', '07744503579', '07800424445', '07828463103', '07929216401', '07965680632', '07993321814', '08185990611', '08239391182', '08263255061', '08551813478', '08951037993', '09200474636', '09305829275', '09309357583', '09395234399', '09567087115', '09590752511', '09617715600', '09658199637', '09772390010', '09781217704', '09904540086', '10131485190', '10303827804', '10381883145', '10387955275', '10530072996', '10682927670', '10751905644', '10756763473', '10763267553', '10763267553', '10832879871', '10843567849', '10846465264', '10868992814', '10869153676', '10891336258', '10899104277', '10899237495', '10902725227', '10902725227', '10942862160', '10962251547', '11009868229', '11009912812', '11038554118', '11042310571', '11058208212', '11059478953', '11064341604', '11209762429', '11212690300', '11228817334', '11253193921', '11315424604', '11362757949', '11417438935', '11417505318', '11425212400', '11439193044', '11457070838', '11464841854', '11481772252', '11489931613', '11518585859', '11535801836', '11557806464', '11586731674', '11610747826', '11625877971', '11625945861', '11626166842', '11732462435', '11740852343', '11756129659', '11782687287', '11789970620', '11820882044', '11853170446', '11856784597', '11894479808', '11913356948', '11922342231', '11929680993', '11942307746', '12054893039', '12134401088', '12136625928', '12170770224', '12174482542', '12177676065', '12179121939', '12242157201', '12257146637', '12284667870', '12309082422', '12362951953', '12391096851', '12391734049', '12398687101', '12439522146', '12518595365', '12599359305', '12614607131', '12684809892', '12797523094', '12925566175', '12949490402', '12956504479', '12977802605', '13214841624', '13284072140', '13387763896', '13402580317', '13421813725', '13429972785', '13430437320', '13485872090', '13509657772', '13546195163', '13576954502', '13587126981', '13595052671', '13665943493', '13775217780', '13845246877', '13850059657', '13949077604', '14023197214', '14091223928', '14153158835', '14141100788', '14193803963', '14285330351', '14418543331', '14470659269', '14606454455', '14607421142', '14695584820', '14757508696', '14824383474', '15030937530', '15230830021', '15284038863', '15369904397', '15521734146', '15671593595', '15770364077', '15767980936', '15843986418', '15945635204', '16134582251', '16203211301', '16423737870', '16648167300', '16742339482', '16869059732', '16916257294', '17302857460', '17318033046', '17406968799', '17545526389', '17852397012', '17978997994', '18131919261', '19547574224', '20036866734', '20309622421', '21445033522', '21608087853', '21734007521', '25078338877', '25716828583', '36187484287', '74480539357', '93760199360']
snils_list=['04042638115', '02827722561', '02638842063', '07478402489', '14220385012', '11211300357', '11929630877', '06363151646', '06363151646', '13257517553', '10553799465', '02640184623', '02640184623', '07989157245', '07010184200', '07989157245', '07010184200', '15330943950', '05617220440', '15389844114', '14300632100', '04478907495', '12339450847', '02673977186', '15330943950', '05617220440', '15389844114', '14300632100', '02453853041', '02829337467', '13990181988', '02674049049', '15155736968', '15534494779', '15155736968', '02674049049', '02711612107', '02634982968', '11241416899', '02711612107', '02634982968', '19698086051', '19698086051', '11059328328', '07868082620', '07868082620', '03354606737', '12406317011', '03354606737', '12406317011', '02721648437', '15279677310', '15279677310', '13400364806', '13400364806', '02644279048', '02644279048', '17929250713', '17929250713']
snils_list=['025-797-830 95', '081-813-358 62', '025-793-199 79', '108-440-912 39', '026-474-962 73', '082-378-683 99', '026-342-725 33', '084-884-904 25', '074-585-224 87', '027-119-156 30', '128-161-842 54', '120-228-206 91', '119-682-837 00', '182-197-563 95', '192-500-717 54', '026-482-619 58', '119-635-948 93', '197-787-891 81', '140-119-435 10', '059-837-512 11', '119-585-764 05', '027-481-040 40', '137-036-615 45', '040-426-390 16', '142-203-850 12', '028-277-225 61', '094-426-442 79', '024-538-530 41', '128-161-842 54', '119-682-837 00', '127-427-751 67', '059-476-226 00', '026-655-456 66', '077-923-892 23', '129-617-399 92', '195-928-743 42', '023-824-039 25', '138-484-811 95', '063-016-801 22', '138-194-360 76', '081-088-791 82', '028-512-301 24', '101-655-183 18', '029-497-315 91', '067-550-147 69', '027-803-908 59', '124-323-189 20', '108-690-072 61', '070-101-544 93', '097-343-903 00', '136-574-511 73', '173-859-329 15', '120-123-736 86', '068-465-945 16', '046-978-554 22', '026-438-984 75', '025-799-206 85', '025-789-256 90', '033-541-525 22', '026-740-107 29', '117-158-864 68', '026-010-400 75', '027-792-186 84', '146-393-940 92', '048-368-694 06', '090-140-316 14', '154-552-130 48', '074-838-159 00', '216-103-860 21', '067-392-685 05', '154-552-130 48', '090-140-316 14', '048-368-694 06', '027-792-186 84', '026-010-400 75', '117-158-864 68', '026-740-107 29', '033-541-525 22', '025-789-256 90', '025-799-206 85', '026-438-984 75', '046-978-554 22', '068-465-945 16', '120-123-736 86', '173-859-329 15', '136-574-511 73', '097-343-903 00', '108-690-072 61', '124-323-189 20', '027-803-908 59', '067-550-147 69', '029-497-315 91', '101-655-183 18', '084-215-414 46', '027-355-301 37', '122-846-987 80', '077-454-001 70', '027-484-462 70', '059-371-050 69', '115-577-822 74', '115-577-822 74', '027-809-003 51', '075-813-199 85', '015-957-993 97', '015-957-993 97', '027-802-894 66', '131-087-354 30', '131-087-354 30', '074-585-224 87', '074-585-224 87', '142-231-953 25', '142-231-953 25', '075-812-741 81', '075-812-741 81', '142-260-491 27', '142-260-491 27', '059-836-997 40', '059-836-997 40', '049-101-066 22', '049-101-066 22', '025-938-246 71', '025-938-246 71', '026-342-115 13', '026-342-115 13', '074-837-621 97', '074-837-621 97', '108-691-985 97', '108-691-985 97', '059-835-580 15', '059-835-580 15', '045-641-379 58', '045-641-379 58', '179-795-676 71', '179-795-676 71', '027-301-953 26', '027-301-953 26', '025-937-181 67', '025-937-181 67', '059-212-114 36', '059-212-114 36', '027-356-633 58', '027-356-633 58', '146-393-940 92', '146-393-940 92', '034-513-255 19', '034-513-255 19', '141-801-527 25', '141-801-527 25', '126-983-708 00', '126-983-708 00', '132-318-242 17', '132-318-242 17', '144-238-387 59', '144-238-387 59', '070-114-837 19', '070-114-837 19', '027-119-031 18', '027-119-031 18', '124-296-857 74', '124-296-857 74', '124-946-910 79', '124-946-910 79', '124-209-478 34', '124-209-478 34', '146-783-790 14', '146-783-790 14', '059-372-159 85', '059-372-159 85', '132-575-175 53', '132-575-175 53', '156-463-602 76', '027-785-889 14', '027-746-365 76', '145-235-548 53', '026-266-289 54', '097-344-088 98', '123-914-707 48', '027-448-632 66', '026-503-459 30', '132-497-274 67', '125-370-241 27', '026-764-810 71', '070-100-363 86', '164-272-778 82', '164-272-778 82', '126-509-195 58', '126-509-195 58', '070-153-772 35', '070-153-759 38', '149-897-110 28', '149-897-110 28', '027-486-216 66', '027-486-216 66', '139-650-471 83', '139-650-471 83', '027-502-771 38', '027-502-771 38', '027-817-728 78', '027-817-728 78', '117-532-151 32', '117-532-151 32', '049-102-263 29', '049-102-263 29', '079-072-717 91', '079-072-717 91', '037-602-013 21', '037-602-013 21', '067-597-788 43', '067-597-788 43', '026-401-433 06', '026-401-433 06', '027-364-353 50', '027-364-353 50', '190-886-605 14', '190-886-605 14', '124-657-103 47', '124-657-103 47', '132-311-297 04', '132-311-297 04', '055-075-524 52', '055-075-524 52', '077-672-978 31', '077-672-978 31', '079-294-525 14', '079-294-525 14', '108-440-880 48', '108-440-880 48', '027-204-176 15', '027-204-176 15', '059-509-081 85', '059-509-081 85', '027-196-702 62', '027-196-702 62', '069-341-694 92', '069-341-694 92', '072-154-327 36', '072-154-327 36', '045-491-076 59', '045-491-076 59', '029-508-242 56', '029-508-242 56', '123-256-565 38', '123-256-565 38', '027-119-857 52', '027-119-857 52', '027-119-518 36', '027-119-518 36', '072-155-672 54', '072-155-672 54', '059-838-348 21', '059-838-348 21', '104-799-864 99', '104-799-864 99', '096-003-242 41', '096-003-242 41', '173-859-329 15', '173-859-329 15', '059-836-306 03', '059-836-306 03', '060-085-287 36', '124-288-241 51', '059-839-874 42', '107-648-644 75', '058-866-714 23', '065-377-203 72', '027-377-144 60', '026-756-256 70', '147-017-663 55', '073-696-072 97', '026-566-775 81', '125-981-374 84', '027-326-453 41', '073-243-535 46', '083-554-259 80', '083-554-259 80', '079-891-817 47', '079-891-817 47', '026-475-770 71', '026-475-770 71', '078-045-931 85', '078-045-931 85', '120-801-660 06', '120-801-660 06', '074-837-682 09', '074-837-682 09', '077-672-671 15', '077-672-671 15', '045-485-804 78', '045-485-804 78', '070-406-282 27', '070-406-282 27', '124-296-857 74', '124-296-857 74', '025-938-294 79', '025-938-294 79', '129-797-748 38', '129-797-748 38', '138-764-746 10', '138-764-746 10', '060-084-994 52', '060-084-994 52', '068-658-336 16', '068-658-336 16', '025-774-880 83', '025-774-880 83', '025-772-588 74', '025-772-588 74', '071-274-403 40', '071-274-403 40', '027-470-603 44', '027-470-603 44', '108-692-202 61', '108-692-202 61', '046-646-199 83', '046-646-199 83', '025-768-869 99', '150-646-249 51', '026-437-917 60', '027-214-805 26', '138-296-423 88', '118-964-791 11', '121-783-265 48', '164-487-744 08', '027-225-519 32', '027-225-519 32', '040-626-838 39', '040-626-838 39', '093-595-158 07', '093-595-158 07', '068-465-602 97', '068-465-602 97', '027-330-114 06', '027-330-114 06', '040-641-143 05', '049-510-435 52', '120-914-143 13', '059-835-336 05', '228-268-227 80', '227-143-165 40', '107-037-160 15', '174-863-443 04', '174-863-443 04', '107-648-626 73', '107-648-626 73', '068-080-514 64', '068-080-514 64', '138-733-287 86', '138-733-287 86', '080-086-394 58', '080-086-394 58', '026-475-820 64', '026-475-820 64', '151-098-249 55', '151-098-249 55', '078-680-561 14', '078-680-561 14', '076-533-514 75', '076-533-514 75', '104-616-219 18', '104-616-219 18', '118-791-797 08', '118-791-797 08', '027-246-934 57', '027-246-934 57', '026-501-411 06', '026-501-411 06', '027-119-049 28', '027-119-049 28', '026-133-726 21', '026-133-726 21', '079-511-630 81', '079-511-630 81', '042-603-261 12', '042-603-261 12', '027-204-077 13', '059-837-981 36', '026-007-949 29', '025-951-548 64', '114-952-082 49', '151-298-504 63', '116-982-086 82', '116-810-956 54', '027-339-895 80', '127-205-860 41', '059-837-686 32', '025-991-636 83', '107-675-045 61', '070-101-539 96', '045-641-381 52', '126-848-374 93', '080-234-740 35', '026-652-528 53', '027-120-028 93', '129-899-077 36', '025-908-261 55', '026-440-781 39', '026-507-843 50', '026-482-427 52', '117-184-682 63', '102-573-632 25', '226-852-954 97', '125-432-346 29', '011-557-974 42', '159-765-149 22', '026-434-514 33', '026-475-918 73', '121-176-516 19', '027-481-539 62', '027-745-664 80', '040-680-558 40', '070-099-872 76', '176-867-177 35', '073-204-418 26', '144-921-302 47', '136-970-871 01', '078-681-074 08', '125-185-677 64', '026-013-559 08', '026-013-559 08', '070-101-539 96', '045-641-381 52', '126-848-374 93', '080-234-740 35', '026-652-528 53', '027-120-028 93', '129-899-077 36', '025-908-261 55', '026-440-781 39', '026-507-843 50', '026-482-427 52', '117-184-682 63', '226-852-954 97', '125-432-346 29', '011-557-974 42', '159-765-149 22', '026-434-514 33', '026-475-918 73', '121-176-516 19', '027-481-539 62', '027-745-664 80', '040-680-558 40', '027-303-350 13', '026-482-527 55', '102-573-632 25', '027-119-109 23', '027-119-109 23', '027-196-196 66', '027-196-196 66', '202-684-277 50', '202-684-277 50', '075-974-198 23', '075-974-198 23', '026-342-110 08', '026-342-110 08', '221-707-124 21', '221-707-124 21', '141-430-538 15', '141-430-538 15', '105-580-590 46', '105-580-590 46', '095-617-461 00', '095-617-461 00', '026-441-518 30', '026-441-518 30', '059-839-031 07', '059-839-031 07', '081-813-349 61', '081-813-349 61', '142-471-713 43', '142-471-713 43', '114-024-227 88', '114-024-227 88', '073-107-439 37', '073-107-439 37', '067-237-706 78', '067-237-706 78', '027-800-633 39', '027-800-633 39', '027-484-751 76', '027-484-751 76', '084-303-149 41', '084-303-149 41', '059-836-044 00', '059-836-044 00', '068-859-164 30', '068-859-164 30', '219-524-554 73', '219-524-554 73', '033-604-934 33', '033-604-934 33', '119-692-556 99', '119-692-556 99', '148-970-137 00', '134-145-851 41', '073-204-224 18', '027-468-680 84', '027-483-690 76', '076-006-844 57', '073-084-371 56', '070-137-861 41', '026-478-572 79', '058-639-594 18', '153-084-167 47', '138-194-343 75', '027-792-186 84', '027-755-276 77', '068-962-726 25', '027-340-485 35']
snils_list=['02744295861', '07514917473', '13665297387', '02780636360', '02718867983', '08856113092', '02675583478', '06008494846', '05815539881', '02234872630', '02635760551', '16431357856', '13781690994', '02747069059', '13289363681', '07483779315', '16407276666', '02092801013', '02746731158', '06537756696', '07324292552', '04059282143', '02733664047', '12445157034', '02720375524', '15114223809', '14435549263', '07432264449', '14845972106', '11962374470', '02949307174', '01746917063', '07751423774', '12775892204', '07933052575', '10128639119', '02830569247', '06739268505', '09415005141', '06723776993', '02600956121', '07982799452', '18466402692', '12874342373', '02780415445', '14319939378', '04549108560', '02718728868', '00805727634', '02268291648', '11232067697', '15689804426', '10949153772', '14906591688', '02600563809', '02950810851', '02951294258', '04103860508', '09561273688', '11342812212', '02650217921', '08180781577', '02780601442', '08019109844', '09664218400', '13858516194', '02579586700', '10671890157', '09676924741', '02748208759', '02656677581', '12273217626', '04064114305', '09676918746', '05983939027', '02674129148', '04549103146', '02626424635', '10964678492', '06253426444', '13624610841', '05832976496', '01933919161', '15118010307', '12028279023', '13498857923', '16465049580', '02629418655', '02657192259', '02676100134', '02676019353', '07040214301', '07320422418', '05886670926', '02806453038', '01601050474', '16685927127', '04132323394', '06973805311', '12284132124', '02676342158', '17667740218', '12953533168', '04260313911', '06723858490', '02949734496', '10869198800', '11812733132', '01552346725', '17686717735', '11650374938', '02735428550', '04185718260', '07118573560', '12616244939', '02835468569', '14289054575', '02676272870', '02736163040', '02734092033', '04862344671', '05308658156', '01309874539', '11727880876', '05506915962', '02675445971', '02578824886', '02676481071', '07894669347', '07320390532', '11033993018', '02647749074', '11313917920', '14897013700', '02734900139', '11626159744', '02951249051', '06537720372', '12711532219', '14197537079', '13915202745', '14220371304', '01519689968', '12744647471', '13826306357', '02176886264', '05328758677', '02664725360', '12716925364', '05141675737', '02734581856', '02657121539', '14011950002', '02737517155', '15074662157', '04419684369', '02649515160', '11273074824', '12332582930', '01302484996', '02126776732', '09734463401', '16255306251', '07989169252', '05815142955', '11503559423', '14956510793', '02774618877', '14571558780', '11489686804', '12166219935', '08586515109', '12403380104', '01747047647', '14905139867', '01592443444', '12775766906', '02650345930', '02735044027', '14684651697', '08168031663', '12745299075', '15008964656', '06220602001', '11285105724', '08951044687', '02335028304', '14753313964', '08125526449', '07535782493', '06103844223', '02950921557', '02780249656', '12178326548', '12402470000', '02601044689', '07320420616', '12402507097', '12149050419', '12391772966', '12343407317', '07084596183', '02781429865', '02664363857', '14643763376', '05983707006', '03762843674', '02748727884', '02778774002', '02594718778', '12402543202', '13719901575', '01850688461', '11062473005', '13751836877', '00891145545', '12775751792', '02731979567', '04125975951', '07756779737', '02726382248', '17320474648', '05983987442', '02719415044', '02675874994', '13176937777', '02656432753', '09415065462', '06402917240', '12951941476', '12583594784', '04924102236', '02778156077', '02600794929', '05142469638', '02634623435', '06921789198', '12170796242', '03775647089', '12055159925', '12204989844', '07744283995', '11929615275', '01848871890', '02600976834', '11420677217', '02778591196', '06906102957', '12920088039', '02676064762', '13529748484', '13496853501', '08263220244', '09561534892', '13815631862', '01590981765', '02577863386', '13328190541', '07868116712', '11926974490', '05984046195', '07215385955', '07595759534', '07503197252', '02789037181', '14943128573', '11891714373', '12583914776', '04054358125', '04607193247', '10980967693', '02748442666', '12391470748', '03638911771', '02801209204', '01031761180', '11027121581', '02599050670', '03360634021', '07597387634', '02676441059', '00269356336', '11347571342', '11009873222', '02074182817', '20806963467', '08567502194', '05831703254', '09664183406', '02779563701', '13781691390', '02236322504', '05713901451', '13807658987', '07535674793', '12015665512', '05713956779', '02778020052', '07013762228', '02729736878', '02836093254', '06789870142', '17927754832', '02672277961', '03867149791', '12058913948', '02676931379', '12748205055', '18720703471', '13207882445', '08675998252', '07320298039', '12831012515', '10033818396', '02778711786', '08561499403', '02382403925', '15646360276', '11626172837', '11625865055', '13120845914', '02657097467', '01590255635', '04894077400', '02629755875', '11565815057', '07462873189', '16158483382', '11591033126', '01305340886', '13502822423', '02776046969', '03765984606', '06162134425', '13661102326', '06220693535', '02779933607', '06755179498', '12802117617', '17481151470', '09734382199', '13120856616', '07010120476', '07264940376', '14115381218', '02339646249', '07929216502', '13972422073', '00840717428', '01912155318', '12015916006', '17633179082', '10200828481', '02860743255', '05562727270', '15064624951', '05983981531', '10660853948', '14636510457', '13382234339', '11963594893', '07221189739', '02675605260', '02734688369', '03273606737', '12428824151', '02743576561', '05563070046', '07326491164', '01604592121', '12452761949', '01850038826', '13037354020', '06502171726', '07320441826', '04049596968', '02749175674', '02590608549', '11634464644', '02172742725', '11600641501', '17352772785', '01308553312', '01310905384', '05716883490', '02720277019', '06657713293', '04155106312', '19578753550', '01618419840', '06766512594', '07166723674', '02622513109', '13485907285', '11603233804', '09979743975', '16149643983', '13474599087', '02579319979', '15615004433', '02722059120', '11626352334', '02570830740', '06685162290', '01092072703', '04549101546', '02746491467', '11969260490', '02732577956', '16149465177', '02743674662', '11086471235', '17217461762', '02339603231', '06872498619', '16690130469', '20545621437', '00903156412', '02582583462', '02219018201', '18512430351', '02789943818', '10919674681', '04649542179', '09617866520', '02579445176', '21229221921', '12391335136', '02609876471', '11940981267', '07466865715', '02780900351', '07010110776', '12915656978', '02676903778', '02255602820', '14020900695', '12178739270', '06545368582', '01309860629', '15571664585', '10826904050', '06779523216', '06008528736', '02600993733', '05937005772', '12720584845', '04663827480', '13231129805', '11958576405', '02779517492', '02719738270', '07009987276', '01306470400', '05282478569', '12332813822', '08504351451', '21036901314', '02626577660', '02613490425', '02740242315', '14020900493', '11400541887', '07539115373', '16313917556', '11941942969', '02727186455', '02648089568', '02748508064', '02779655000', '12866570895', '20652374444', '11557710451', '02720295930', '02656565469', '11817173549', '12302435899', '19917902528', '18154994908', '20494710959', '07834734397', '02722058421', '14261870250', '14039586766', '03360372926', '02780212532', '11220843901', '11724036523', '13007983841', '13226547337', '04564115039', '02729677585', '02835464056', '16048148456', '13799499141', '10950527142', '11341699843', '02720198528', '11929273172', '14724184663', '07406519761', '05983733613', '13703661545', '05506850960', '02598857509', '02748183264', '02826325243', '10821895656', '06491271772', '04697956428', '05506097954', '09941001873', '11325704717', '18094217980', '17868697262', '02576916881', '07029351142', '07459341084', '14235325732', '06906097384', '02675896499', '07535855393', '02577981392', '13398102156', '15646359796', '02734166844', '13120846815', '02736722959', '12308267328', '05947622600', '10693147958', '11406596336', '02676390472', '02748104040', '03760290750', '02664331339', '11800955235', '13885518100', '04336046731', '02778396501', '10764864475', '02778164783', '12990447992', '09734408898', '14620726343', '02744837265', '06789916645', '02861604448', '13231801104', '15681621681', '02775578795', '13751246048', '11090140592', '11510854012', '04059190441', '01303101656', '11094910736', '13697087101', '07792389223', '02775347271', '14701766355', '11707147232', '05712483555', '07828127592', '14193974182', '02523807831', '04910203923', '15629516183', '15494201969', '04951058161', '02656502445', '07369607297', '09511933573', '02949357694', '11464346541', '13148964671', '06846339803', '07643288791', '05051504709', '13819436076', '11713915641', '01591658157', '02950052532', '02743758567', '14355925472', '08360312443', '07459512085', '05831959390', '01665378463', '02672452044', '15208686062', '06921826481', '07929513508', '08108879182', '12710616826', '12920772355', '04923718876', '16774507808', '12046892546', '14070935946', '11767635483', '13382300528', '07310738843', '02779558809', '07535735282', '14613853359', '14162472337', '16383886714', '02254532618', '10651904736', '02781559777', '14828507284', '11325231801', '07459246898', '13546188065', '13848481195', '02626628954', '02748849595', '11481782154', '02457683769', '06545538177', '08140241824', '06807899115', '08019069656', '02594053752', '02591518351', '02336806836', '02072481817', '15371247454', '13829642388', '05937221778', '13644106337', '02255380626', '11240347497', '14783691206', '14739366397', '02657085763', '02834851866', '02736411942', '20556919473', '14410220392', '01552804022', '13609194462', '04862138971', '02949270379', '12502735521', '13019614927', '17046784482', '06871818606', '01745544247', '06766465308', '10962070139', '11941594871', '06402967154', '02729530757', '11231394505', '01565587869', '02732530122', '06807964807', '06808039171', '13431807029', '10128963128', '06253573255', '14802950158', '05527088465', '17110440510', '13921942771', '08192785601', '14717304252', '14907954198', '08525625075', '13005060783', '15985283930', '15370398167', '09635894123', '15120404395', '17639119902', '16395228187', '11535094534', '14796023792', '09512302240', '19758401824', '12148056224', '12324622718', '13264253940', '06008546435', '12307378332', '12360374426', '15488606000', '12990104962', '21065895964', '11304131376', '22377674993', '14756715292', '10558185758', '09676822026', '12775872602', '02719408653', '10949723989', '02626841146', '12530927740', '16998171039', '09561295900', '05507412840', '13845313058', '11019695028', '09941351800', '14729174786', '14856224583', '06197821487', '13778952118', '14123556627', '09979856988', '07242878678', '13015192301', '13447990587', '11703913833', '11038540107', '08766424411', '10812848949', '12093530830', '14234546036', '10873657579', '06984788348', '02336876958', '07982729431', '07086406061', '11941912455', '12708450751', '02746868084', '02725204017', '13853395082', '11433623013', '02598928102', '02739239062', '13198542978', '11431844827', '02826340441', '11499541077', '02640092113', '12961739992', '02646876884', '02735578367', '08110425109', '12428557861', '14485787410', '08025770352', '11225891129', '02737714460', '02664420843', '02647645769', '12411297719', '13366360756', '12608151229', '12921385757', '02775644378', '02711961738', '12537024127', '07483750897', '05806491777', '01858504768', '09590580609', '17649701913', '09512161248', '12231687023', '01304756104', '15021818824', '11387937786', '12255190428', '02729045952', '02830139424', '12189197473', '02728106133', '14240153003', '05510823024', '01618795666', '10775771883', '04910210617', '11013340769', '02722925240', '04563501645', '01850782554', '02863343254', '02749639387', '02834851058', '07320372227', '02672568871', '09720282573', '02850775262', '07029124333', '11535317528', '11585967092', '05246198862', '02674173454', '02774538576', '05505537547', '12285309243', '11203620985', '11507352623', '12805020112', '05506217635', '18565759028', '02673926876', '01870131727', '14318778678', '16101967743', '01304746707', '07040080710', '11006898627', '06952216982', '02750420016', '08263218762', '13349152348', '12754231042', '01857070958', '13793070374', '11879062485', '02646451650', '01466049839', '02598557396', '20954198784', '10871473054', '13363625044', '12203791515', '07147435763', '04059303529', '06984796044', '09941022881', '02074778749', '10613351505', '11406615819', '06492105457', '04549247877', '04335462336', '09561261883', '13282028531', '11045886235', '07323494559', '02811738742', '13878078107', '06593219888', '08263269072', '14800624842', '00807634233', '06022688134', '02595017854', '02850534846', '19298731228', '11039409215', '11240277805', '04031620388', '04131524904', '03354030407', '07310741024', '14110928213', '03357127437', '12610502400', '07791379215', '12416807034', '02336141002', '11323622503', '02646961673', '13664678900', '09988201738', '06491512263', '02664899393', '12963185379', '18932347101', '11203615992', '02336794047', '14865860208', '05809958511', '02308141600', '02667581579', '20049547133', '10963840268', '10353792942', '13857891013', '10718329847', '01308756326', '08019038241', '10851962259', '02582501131', '14005266407', '06470466467', '16467011464', '02182753938', '01998162898', '02782006236', '11830818041', '06253859576', '13731710540', '12258340739', '07906792511', '07010022173', '04875538200', '10693164554', '12923119847', '01732391227', '13457134046', '01381154917', '06194687595', '02371959557', '13614460741', '12006154082', '08834846113', '11472771956', '01961970068', '12416857251', '11007254487', '06685078907', '12043557214', '13583641169', '02626227128', '03353049019', '03352963650', '08125514442', '01518631326', '02272726633', '02621887953', '02676171864', '02609754760', '02451426517', '12965860805', '12307381624', '02629127947', '19012476348', '02600977230', '10789830192', '11288452659', '07744523282', '14230031286', '12300302264', '15614994289', '13350730121', '03354457140', '02742846862', '01036307991', '09664154399', '12816192354', '06489483720', '10204986216', '04683063464', '12402520493', '02613843937', '02291477948', '07643170465', '07249344777', '13495638189', '16741031651', '18103228731', '11926978094', '08031022804', '02728958992', '02776807385', '00805921125', '01316438308', '02830459846', '11479721067', '08110361614', '02948946001', '01553348431', '02826013527', '01321484498', '10723297033', '15694414596', '06706988405', '11065800312', '02830719947', '10505144598', '01527324318', '03765566891', '06252578159', '06492303966', '07491228072', '01732835035', '01598296386', '19764701822', '05509465669', '02366896174', '20013278993', '18588936253', '15001965522', '13624721547', '01997811803', '10558192553', '06778465627', '12607962362', '01300278276', '02948975715', '00682095642', '07458743907', '16808970311', '01163730709', '12550769154', '02453800424', '15634125553', '09590464809', '15572234060', '12693898918', '15076195263', '08025639859', '02273638944', '14149190148', '02578130550', '02598761193', '12174884158', '10951536451', '02183173622', '01343441805', '02725359551', '11853156351', '13973532388', '02790092049', '05984397426', '11406608923', '01589298288', '01745826156', '02318851536', '07946643821', '04067640543', '12827898405', '14041520601', '01845153340', '03577491080', '04294074351', '12539328965', '01686705974', '02613491023', '01597550268', '02651562640', '02599542994', '10553928349', '02635690758', '14026376229', '07493843304', '14583207363', '13352606029', '03356439350', '14519146761', '03765392177', '02711434311', '14303938946', '00890939173', '07975919243', '11942344146', '07470713264', '04632621538', '11406590021', '13833117648', '13733065042', '21617220830', '12583775081', '13799044908', '13107729632', '05863959317', '12775919703', '02781185560', '09432701158', '11976204067', '07804045760', '15138406546', '02646961370', '10873654068', '02453673645', '10942779072', '11305779433', '02591335141', '11879210070', '02171761623', '13547623870', '02776833487', '12861537367', '11460436926', '18076645696', '02074005694', '09676837039', '02711327714', '02171758230', '11205618099', '02850691458', '09980053709', '02274865253', '12270144812', '05831915572', '11389593992', '01092010075', '12298615675', '11212690300', '02171769538', '07274435066', '01711207999', '11733564044', '03775415371', '02749626782', '04025450908', '10400738191', '04892367903', '01915858468', '05511300401', '06746085791', '07406735969', '18594486941', '13895954227', '13527363555', '07029257756', '05508924872', '02485481664', '11681120223', '11051846613', '05505485756', '11747280864', '09937789056', '10796269489', '08505372666', '06918123678', '09677249128', '07929498134', '02074961538', '02712150000', '05562695788', '06253880163', '04564539271', '01662693254', '02337501920', '02728311841', '07745513486', '12391203826', '02338725947', '07850100550', '13975708914', '02230170881', '14875031478', '11966934300', '13657536888', '13441475238', '02667173059', '14587372499', '12017320993', '03451280220', '02074508619', '09032388657', '13108184520', '05510221602', '14157107030', '04067874467', '07982792943', '06491214255', '11417430515', '01310733482', '02576886999', '13638950588', '13215185116', '05509844980', '06161632335', '02540451108', '08219101236', '02728809268', '02832374242', '18821710480', '07275726587', '02788723698', '13743035140', '06826910993', '07661639594', '15135404224', '07858925234', '04117511711', '02171658125', '01685882991', '11943254453', '05757492807', '03867166892', '02337571739', '06984352109', '02590569262', '12429684166', '12003967714', '02292835751', '15231255119', '08108534350', '01703227201', '02868966921', '02171509815', '12464082339', '01517960441', '01310343772', '02454610727', '07092061948', '13667273080', '15151871651', '02732240719', '11896503991', '12496470477', '11955464177', '10505647018', '13902117935', '15814562880', '20058959258', '02307420902', '11941096350', '11542543730', '13610329723', '13427519149', '05506698172', '06491754085', '02181644322', '15835725899', '02486850876', '02787186794', '17710698894', '07185473987', '13448097671', '17007784362', '19140631248', '04844247570', '11209746431', '13482215948', '12920872662', '12545876679', '02595736482', '11626093033', '03360756641', '13920645861', '11048353215', '02190929440', '04725025942', '04915135149', '02593167661', '10989090293', '10889747414', '12949271794', '02863090958', '08107885980', '14803165952', '02811844034', '13612546034', '10899065899', '02218732723', '10982832882', '02951479270', '07218047344', '12203641597', '12539249868', '04294185966', '13935604777', '12921452746', '14770902374', '11941560551', '01746521442', '00675873774', '17012149822', '02951597781', '07078235670', '11279744166', '11430769228', '15480024846', '10291045205', '14286035556', '12481514743', '07029193958', '06491819087', '12545865169', '07946594228', '12203291697', '07965832730', '17053271545', '13649605380', '09808485319', '05992376515', '02788970715', '08452472880', '05833178574', '13434638350', '15723126450', '08563949519', '01744596564', '01093151906', '06922630974', '02183140001', '12949625500', '11454891459', '02951205940', '07894725634', '02595766491', '02074901621', '08005647644', '01037659926', '02340302279', '06254229548', '05576710380', '07407231340', '11402873922', '13302955327', '08164011435', '12651684057', '14172577155', '02381656242', '02711667940', '01410202155', '02950024628', '17710660566', '10933709252', '06579919732', '12476813368', '02593087158', '13426446749', '08586396430', '11633783857', '15140836234', '02635348747', '01302747093', '13985853323', '05509698187', '00889553695', '07966436929', '11286208032', '13506173836', '05832602152', '05539614981', '08511080538', '02728087256', '02722112805', '01035112870', '12797509201', '14757507189', '12802101804', '02738047856', '02827724767', '08551786905', '11426103602', '06754859111', '01648006631', '12994912609', '14625429261', '12043423799', '02254812826', '01663030918', '07895117414', '12667462479', '02648871076', '01302098688', '07711341142', '09289807326', '02774826379', '06945356805', '13686433588', '01590215118', '01309838333', '06905837191', '07906771503', '11817869788', '13909847401', '06606600144', '02643738352', '13714623649', '14209765762', '09734471602', '04068513238', '05510805325', '12632758760', '07858940937', '11307836025', '12775861395', '17326060247', '05284503148', '03572917063', '12022125673', '01308139000', '13046551528', '19078394917', '13593237370', '18850965425', '11952900863', '03951763073', '02207249810', '08676592431', '12286820761', '00803430803', '02834912153', '11265071317', '03775490789', '11949862607', '14305543024', '20569354674', '14465051147', '02190772332', '13840198059', '12194765263', '15355911368', '15964267915', '12630664236', '02453353627', '12362246023', '02775317363', '11358711848', '11226030082', '05092275143']
snils_list=['05983962426', '06492527281', '09590550600', '11251084200', '12797523094', '12830972564', '13276386369', '14153158835', '15813447774', '00261120204', '00803365212', '00804142399', '00806129013', '00808829354', '01032109260', '01067124703', '01157677644', '01162245091', '01245485020', '01308248206', '01310196987', '01310441772', '01310580281', '01315677160', '01324037488', '01344928939', '01346747339', '01382344624', '01382527834', '01384738251', '01433122992', '01447082447', '01486471759', '01487642563', '01552516926', '01588346975', '01592199156', '01649274257', '01649626565', '01653951047', '01693051035', '01733447839', '01733566443', '01745728661', '01847654069', '01847654069', '01849334768', '01849334768', '01849465985', '01850576349', '01855897091', '01910103188', '01910393425', '01997848018', '01997895128', '02001765582', '02001829178', '02004158677', '02004158677', '02004172671', '02071520695', '02074005694', '02075495136', '02092854842', '02126878033', '02171240494', '02171748025', '02171767837', '02172896449', '02174647135', '02177268142', '02178341434', '02212512275', '02253765939', '02255058318', '02255405515', '02255687753', '02259466555', '02268151026', '02268171032', '02270344719', '02274881453', '02308584836', '02316856637', '02336197433', '02337453228', '02339318636', '02339695363', '02340457207', '02372403919', '02375581251', '02376194047', '02392410322', '02484012523', '02520441194', '02520441194', '02523557329', '02540446519', '02540919233', '02572038027', '02576694687', '02576694687', '02578015146', '02579491082', '02579562382', '02582484157', '02592810958', '02593048350', '02593125241', '02593483671', '02595100437', '02599022059', '02600491305', '02605421826', '02610925318', '02613412001', '02623598534', '02623923133', '02624347431', '02625344733', '02629031126', '02629156752', '02635342230', '02643791760', '02644246437', '02644716044', '02645458558', '02645631445', '02645748466', '02645912352', '02646421944', '02648497886', '02651194536', '02651862753', '02653964266', '02655866074', '02656012731', '02656166859', '02657132039', '02657363967', '02661122935', '02664193959', '02664507350', '02664773876', '02667724373', '02667738687', '02675469480', '02677161462', '02711523714', '02711707623', '02711952131', '02718734863', '02722720929', '02724182028', '02727119945', '02727488572', '02729075961', '02735908059', '02736412439', '02736412439', '02736932171', '02737652969', '02738604458', '02739439878', '02746760872', '02748200945', '02748727985', '02749351466', '02749569291', '02776600666', '02779440887', '02781767582', '02801509317', '02806440635', '02811957077', '02811964650', '02827769688', '02831293742', '02831538039', '02833485862', '02850381847', '02868922093', '02951407548', '03288018659', '03354229430', '03354473643', '03357954775', '03363112410', '03641251221', '04005562503', '04032434396', '04059240228', '04124990134', '04133294418', '04260407011', '04284338454', '04329390048', '04371380843', '04459385585', '04547183362', '04615969581', '04632641643', '04725195260', '04885358911', '05010113418', '05051479534', '05051503909', '05265673873', '05283065347', '05506313631', '05506315938', '05507454452', '05507548360', '05522113312', '05639470384', '05757479815', '05809877814', '05810180433', '05810550945', '05815422477', '05831915572', '05937138892', '05983450400', '05983880828', '05983900808', '05983960018', '06187484287', '06220718222', '06220776135', '06253219136', '06253924965', '06254880774', '06441672457', '06624172858', '06714827579', '06759783735', '06807903713', '06841874497', '06859065913', '06889962854', '06952181989', '06984739436', '07029238045', '07031249117', '07031280111', '07076645982', '07092957756', '07095992202', '07118623448', '07122958657', '07199594015', '07210313804', '07250282136', '07255107547', '07269141270', '07271135066', '07291626676', '07309582069', '07328437572', '07372825682', '07406746166', '07407207242', '07407207242', '07417926987', '07458952916', '07459546405', '07478243697', '07483778515', '07502993473', '07553424368', '07573091783', '07597423814', '07610856971', '07661467793', '07661668298', '07681048886', '07744304876', '07744503579', '07800424445', '07828463103', '07929216401', '07931944097', '07965680632', '07993321814', '08032856858', '08109719266', '08120565333', '08185990611', '08239391182', '08263255061', '08551813478', '08951037993', '09127308154', '09200474636', '09305829275', '09309357583', '09395234399', '09425282375', '09567087115', '09590752511', '09616308077', '09617715600', '09658199637', '09772390010', '09781217704', '09904540086', '10131485190', '10303827804', '10381883145', '10387955275', '10530072996', '10682927670', '10751905644', '10756763473', '10763267553', '10763267553', '10832879871', '10843567849', '10846465264', '10868992814', '10869153676', '10891336258', '10899104277', '10899237495', '10902725227', '10902725227', '10942862160', '10962251547', '10986425788', '11009868229', '11009912812', '11038554118', '11042310571', '11058208212', '11059478953', '11064341604', '11069471036', '11209762429', '11212690300', '11228817334', '11241510689', '11253193921', '11315424604', '11362757949', '11417438935', '11417505318', '11422381913', '11425212400', '11439193044', '11457070838', '11464841854', '11481772252', '11489931613', '11518585859', '11535801836', '11557706359', '11557806464', '11586731674', '11610747826', '11625877971', '11625945861', '11626166842', '11689324683', '11732462435', '11740852343', '11756129659', '11782687287', '11789970620', '11820882044', '11853170446', '11856784597', '11894479808', '11913356948', '11922342231', '11929680993', '11942307746', '11969417295', '12054893039', '12115725814', '12134401088', '12136625928', '12170770224', '12174482542', '12177676065', '12179121939', '12242157201', '12257064837', '12257146637', '12284667870', '12309082422', '12362951953', '12391096851', '12391734049', '12398687101', '12439522146', '12518595365', '12599359305', '12614607131', '12684809892', '12925566175', '12949490402', '12956504479', '12977802605', '13214841624', '13284072140', '13387763896', '13402580317', '13421813725', '13429972785', '13430437320', '13485872090', '13509657772', '13546195163', '13576954502', '13587126981', '13595052671', '13595518691', '13665943493', '13775217780', '13845246877', '13850059657', '13935034863', '13935034863', '13949077604', '14023197214', '14091223928', '14141100788', '14193803963', '14285330351', '14418543331', '14470659269', '14606454455', '14607421142', '14695584820', '14757508696', '14824383474', '15030937530', '15230830021', '15284038863', '15369904397', '15521734146', '15671593595', '15770364077', '15796798093', '15843986418', '15945635204', '16134582251', '16203211301', '16423737870', '16648167300', '16742339482', '16762321172', '16869059732', '16916257294', '17302857460', '17318033046', '17406968799', '17545526389', '17852397012', '17978997994', '18131919261', '19008007640', '19489035928', '19547574224', '20036866734', '20309622421', '20719974587', '21445033522', '21608087853', '21734007521', '25078338877', '25716828583', '36187484287', '74480539357', '93760199360']


In [14]:

name=''
patronymic = ''
surname=''
birthday=''
police=''
snils=11966934300
if snils!='':
    if type(snils)==str:
        list_snils = re.findall(r'\d+', snils)
        snils = ''.join(list_snils)
    if type(snils)==int:
        snils="{:011d}".format(snils)
    snils=snils[:3]+"-"+snils[3:6]+"-"+snils[6:9]+"-"+snils[9:]
params={
    "PersonFirName_FirName": name,
    "PersonSecName_SecName": patronymic,
    "PersonSurName_SurName": surname,
    "PersonBirthDay_BirthDay": birthday,
    "Polis_Num": police,
    "Person_Snils": snils,
}
ecp_search_person_resp=session.request("POST", baseurl + '/?c=Person&m=getPersonSearchGrid', data=params)

In [15]:
ecp_search_person_resp.request.url

'https://ecp.mis66.ru/?c=Person&m=getPersonSearchGrid'

In [18]:
ecp_search_person_resp.json()

{'data': [{'Person_id': '660910001134339',
   'Server_id': '13003640',
   'PersonEvn_id': '82547418',
   'Person_IsInErz': '2',
   'Person_Phone': '9920106275',
   'Person_Inn': None,
   'Person_IsUnknownSign': None,
   'Person_IsUnknown': None,
   'FamilyStatus_id': None,
   'SocStatus_id': '10000072',
   'SocStatusFed_id': '8',
   'SocStatusFed_Code': '8',
   'PersonCard_id': None,
   'PersonCard_Lpu_id': None,
   'PersonCard_LpuRegions': None,
   'Person_isOftenCaller': '1',
   'Person_IsRefuse': 'false',
   'Person_IsDead': 'false',
   'existsProjectDeathSvid': '1',
   'existsProjectPntDeathSvid': '1',
   'Person_IsFedLgot': 'true',
   'Person_IsRegLgot': 'false',
   'Person_Lgots': 'true',
   'Person_Is7Noz': 'false',
   'UAddress_AddressText': '620109, РОССИЯ, СВЕРДЛОВСКАЯ ОБЛ, Г ЕКАТЕРИНБУРГ, РЕПИНА УЛ, д. 21, кв. ОБЩ',
   'PAddress_AddressText': '620109, РОССИЯ, СВЕРДЛОВСКАЯ ОБЛ, Г ЕКАТЕРИНБУРГ, РЕПИНА УЛ, д. 21, кв. 44',
   'Person_IsBDZ': 'false',
   'PersonQuarantine_IsOn': 

In [9]:
delimeter=";"
for snils in snils_list:
    ecp_search_person_resp=ecp_search_person(snils=snils)
    if len(ecp_search_person_resp['data'])==0:
        print('no pacients found ' + str(len(ecp_search_person_resp['data'])))
        continue
    if len(ecp_search_person_resp['data'])>1:
        print('many pacients found ' + str(len(ecp_search_person_resp['data'])))
        continue
    if len(ecp_search_person_resp['data'])==1:
        print(ecp_search_person_resp['data'][0]['Person_Snils'],
              str(ecp_search_person_resp['data'][0]['Polis_Num']),
              str(ecp_search_person_resp['data'][0]['Person_IsDead']),
              ecp_search_person_resp['data'][0]['PersonSurName_SurName'],
              ecp_search_person_resp['data'][0]['PersonFirName_FirName'],
              ecp_search_person_resp['data'][0]['PersonSecName_SecName'],
              str(ecp_search_person_resp['data'][0]['PersonBirthDay_BirthDay']),
              ecp_search_person_resp['data'][0]['Lpu_Nick'],
              sep=";",
        )

05983962426;6648320846001177;false;КИЧИГИН;ПАВЕЛ;АЛЕКСАНДРОВИЧ;03.11.1976;ГАУЗ СО "ГП № 3 г. Нижний Тагил"
06492527281;6648420834000320;false;ВОРОБЬЕВ;ВИТАЛИЙ;МИХАЙЛОВИЧ;15.11.1975;ГАУЗ СО "Серовская ГБ"
09590550600;6652210848002260;false;НОВОСЕЛОВ;ВАСИЛИЙ;ОЛЕГОВИЧ;01.07.1987;None
11251084200;6653510896000200;false;ПРИМАК;ЛЮДМИЛА;ПЕТРОВНА;03.06.1984;None
12797523094;6656130839001665;false;СИРОТИН;РОДИОН;СЕРГЕЕВИЧ;10.03.1968;ГБУЗ СО "ЦГКБ №1"
12830972564;6652630885000689;false;КАРЕЛЬЦЕВА;ОЛЬГА;БОРИСОВНА;14.07.1963;ГАУЗ СО "ЦГКБ № 3"
13276386369;6655510843000798;false;УПОРОВ;АНАТОЛИЙ;ИГНАТЬЕВИЧ;06.04.1984;ГАУЗ СО "Талицкая ЦРБ"
14153158835;6656110820001031;false;СМИРНОВ;АНДРЕЙ;ВЛАДИМИРОВИЧ;29.03.1988;ГАУЗ СО "Ревдинская ГБ"
15813447774;6650800897001643;false;ТОКАРЕВА;АЛЕНА;ОЛЕГОВНА;02.09.1991;ФБУЗ МСЧ 32 ФМБА России Заречный
no pacients found 0
00803365212;6658430836000209;false;НАЙМУШИН;АЛЕКСАНДР;ЮРЬЕВИЧ;13.01.1965;ГАУЗ СО "Режевская ЦРБ"
00804142399;6651120872000588;false;АФАНАСЬЕВА;ЕВ

In [5]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import pandas as pd
import openpyxl
import os
import csv
from openpyxl.utils import get_column_letter
import threading

class Application(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Фильтрация организаций")
        self.geometry("600x400")
        
        # Стиль
        self.style = ttk.Style()
        self.style.theme_use('clam')
        
        # Переменные
        self.input_file = tk.StringVar()
        self.output_folder = tk.StringVar()
        self.column_name = tk.StringVar()
        self.sheet_name = tk.StringVar()
        self.head_line = tk.IntVar()
        self.prefix = tk.StringVar()
        self.status_text = tk.StringVar()
        self.progress_value = tk.DoubleVar()
        self.columns_list = []
        self.sheet_names = []
        
        self.create_widgets()
        self.protocol("WM_DELETE_WINDOW", self.on_close)
    
    def create_widgets(self):
        # Фрейм для ввода данных
        input_frame = ttk.LabelFrame(self, text="Параметры обработки")
        input_frame.pack(padx=10, pady=5, fill='x')

        # Выбор файла
        ttk.Label(input_frame, text="Входной файл:").grid(row=0, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.input_file, width=40).grid(row=0, column=1, padx=5, pady=2)
        ttk.Button(input_frame, text="Обзор...", command=self.browse_file).grid(row=0, column=2, padx=5, pady=2)

        # Выбор листа (только для Excel)
        self.sheet_label = ttk.Label(input_frame, text="Лист в книге:")
        self.sheet_label.grid(row=1, column=0, sticky='w', padx=5, pady=2)
        self.sheet_combobox = ttk.Combobox(
            input_frame, 
            textvariable=self.sheet_name, 
            state="readonly",
            width=20
        )
        self.sheet_combobox.grid(row=1, column=1, sticky='w', padx=5, pady=2)
        
        # Выбор столбца
        ttk.Label(input_frame, text="Столбец с организациями:").grid(row=2, column=0, sticky='w', padx=5, pady=2)
        self.column_combobox = ttk.Combobox(
            input_frame, 
            textvariable=self.column_name, 
            state="readonly",
            width=20
        )
        self.column_combobox.grid(row=2, column=1, sticky='w', padx=5, pady=2)
        
        # Выбор папки для сохранения
        ttk.Label(input_frame, text="Папка для результатов:").grid(row=3, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.output_folder, width=40).grid(row=3, column=1, padx=5, pady=2)
        ttk.Button(input_frame, text="Обзор...", command=self.browse_output).grid(row=3, column=2, padx=5, pady=2)
        
                # head_line
        ttk.Label(input_frame, text="Номер строки с шапкой:").grid(row=4, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.head_line, width=40).grid(row=4, column=1, padx=5, pady=2)
        self.head_line.set(1)
        
        # prefix
        ttk.Label(input_frame, text="Префикс для названия файлов:").grid(row=5, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.prefix, width=40).grid(row=5, column=1, padx=5, pady=2)
        self.prefix.set('Гепатиты 244-п февраль')
        
        # Прогресс-бар
        progress_frame = ttk.Frame(self)
        progress_frame.pack(padx=10, pady=10, fill='x')
        self.progress = ttk.Progressbar(progress_frame, variable=self.progress_value, maximum=100)
        self.progress.pack(fill='x')
        
        # Статус
        status_frame = ttk.Frame(self)
        status_frame.pack(padx=10, pady=5, fill='x')
        ttk.Label(status_frame, textvariable=self.status_text).pack(side='left')
        
        # Кнопки управления
        button_frame = ttk.Frame(self)
        button_frame.pack(padx=10, pady=10)
        ttk.Button(button_frame, text="Старт", command=self.start_processing).pack(side='left', padx=5)
        ttk.Button(button_frame, text="Выход", command=self.on_close).pack(side='left', padx=5)

    def browse_file(self):
        filetypes = (
            ('Excel files', '*.xlsx *.xls'),
            ('CSV files', '*.csv'),
            ('All files', '*.*')
        )
        filename = filedialog.askopenfilename(filetypes=filetypes)
        if filename:
            self.input_file.set(filename)
            if filename.lower().endswith(('.xls', '.xlsx')):
                self.load_sheets(filename)
            else:
                self.load_columns(filename)

    def load_sheets(self, filename):
        try:
            wb = openpyxl.load_workbook(filename, read_only=True)
            self.sheet_names = wb.sheetnames
            wb.close()
            self.sheet_combobox['values'] = self.sheet_names
            if self.sheet_names:
                self.sheet_name.set(self.sheet_names[0])
                self.load_columns(filename, self.sheet_names[0])
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось прочитать листы: {str(e)}")

    def load_columns(self, filename, sheet_name=None):
        try:
            if filename.endswith(('.xls', '.xlsx')):
                df = pd.read_excel(filename, sheet_name=sheet_name, nrows=0)
            else:
                df = pd.read_csv(filename, nrows=0, encoding='utf-8')
            self.columns_list = df.columns.tolist()
            self.column_combobox['values'] = self.columns_list
            if self.columns_list:
                self.column_name.set(self.columns_list[0])
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось прочитать заголовки: {str(e)}")
            self.column_combobox['values'] = []
            self.column_name.set('')
            
    def browse_output(self):
        folder = filedialog.askdirectory()
        if folder:
            self.output_folder.set(folder)

    def start_processing(self):
        if not self.validate_input():
            return
        
        # Запуск в отдельном потоке
        thread = threading.Thread(target=self.process_data)
        thread.start()

    def validate_input(self):
        if not self.input_file.get():
            messagebox.showwarning("Ошибка", "Выберите входной файл!")
            return False
        if self.input_file.get().endswith(('.xls', '.xlsx')) and not self.sheet_name.get():
            messagebox.showwarning("Ошибка", "Выберите лист в книге!")
            return False
        if not self.column_name.get():
            messagebox.showwarning("Ошибка", "Выберите столбец с организациями!")
            return False
        return True

    def process_data(self):
        try:
            self.update_status("Начало обработки...")
            input_file = self.input_file.get()
            column_name = self.column_name.get()
            output_folder = self.output_folder.get() or 'output'
            sheet = self.sheet_name.get() if self.sheet_name.get() else None
            
            # Загрузка данных и информации о ширине столбцов
            if input_file.endswith('.csv'):
                df = pd.read_csv(input_file)
                col_widths = None
            else:
                wb = openpyxl.load_workbook(input_file)
                ws = wb[sheet] if sheet else wb.active
                col_widths = []
                for col in range(1, ws.max_column + 1):
                    col_letter = get_column_letter(col)
                    col_widths.append(ws.column_dimensions[col_letter].width if col_letter in ws.column_dimensions else 8.43)
                df = pd.read_excel(input_file)
                wb.close()
            
            organizations = df[column_name].unique()
            total = len(organizations)
            os.makedirs(output_folder, exist_ok=True)
            
            for i, org in enumerate(organizations, 1):
                org_data = df[df[column_name] == org]
                clean_org_name = "".join([c if c.isalnum() or c in (' ', '_') else '_' for c in str(org)])
                output_path = os.path.join(output_folder, f"{clean_org_name}.xlsx")
                
                with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
                    org_data.to_excel(writer, index=False, sheet_name='Data')
                    if col_widths is not None:
                        worksheet = writer.sheets['Data']
                        for idx, width in enumerate(col_widths, 1):
                            worksheet.column_dimensions[get_column_letter(idx)].width = width
                
                self.progress_value.set((i / total) * 100)
                self.update_status(f"Обработано: {i} из {total} ({org})")
            
            self.update_status("Обработка завершена успешно!")
            messagebox.showinfo("Готово", f"Создано {total} файлов в папке:\n{output_folder}")
        
        except Exception as e:
            messagebox.showerror("Ошибка", str(e))
            self.update_status("Ошибка обработки!")
        finally:
            self.progress_value.set(0)
    
    def update_status(self, message):
        self.status_text.set(message)
        self.update_idletasks()
    
    def on_close(self):
        if messagebox.askokcancel("Выход", "Вы уверены, что хотите выйти?"):
            self.destroy()
# def filter_and_save_organizations(input_file, column_name, output_folder='output', sheet='Лист1', head_line=1,prefix=''):
#     """
#     Функция для фильтрации данных по организациям и сохранения в отдельные файлы
    
#     Параметры:
#     input_file (str): Путь к входному файлу (CSV или Excel)
#     column_name (str): Название столбца с организациями
#     output_folder (str): Папка для сохранения результатов
#     """
    
#     # Загрузка данных
#     if input_file.endswith('.csv'):
#         df = pd.read_csv(input_file)
#     elif input_file.endswith(('.xls', '.xlsx')):
#         wb = openpyxl.load_workbook(input_file)
#         ws = wb.active
#         col_widths=[]
#         for col in range(1,ws.max_column+1):
#             col_letter = get_column_letter(col)
#             if col_letter in ws.column_dimensions:
#                 col_widths.append(ws.column_dimensions[col_letter].width)
#             else:
#                 col_widths.append(8.43)
        
#         # col_widths = [ws.column_dimensions[col].width for col in ws.column_dimensions]
#         # print(len(col_widths),col_widths)
#         df = pd.read_excel(input_file, sheet_name=sheet, header=head_line-1)
#         wb.close()
#     else:
#         raise ValueError("Формат файла не поддерживается. Используйте CSV или Excel")
#     print(df)
#     # Создаем папку для результатов, если ее нет
#     os.makedirs(output_folder, exist_ok=True)
    
#     # Получаем уникальные организации
#     organizations = df[column_name].unique()

#     # Функция для очистки имени файла
#     def clean_filename(name):
#         # Заменяем недопустимые символы в имени файла
#         return "".join([c if c.isalnum() or c in (' ', '_') else '' for c in str(name)])
    
#     # Сохраняем данные по каждой организации
#     for org in organizations:
#         # Фильтруем данные
#         org_data = df[df[column_name] == org]
        
#         # Очищаем имя организации для имени файла
#         clean_org_name = clean_filename(org)

#         # Формируем путь к файлу
#         if org in vipnets:
#             output_path = os.path.join(output_folder, prefix+' '+clean_org_name+vipnets[org]+'.xlsx')
#         else:
#             print('нет правила для организации:', org)
#             output_path = os.path.join(output_folder, f"{clean_org_name}.xlsx")
        
#         # Сохраняем в xlsx
#         with pd.ExcelWriter(output_path, engine='openpyxl') as writer: 
#             # org_data.to_excel(output_path, index=False)
#             org_data.to_excel(writer, index=False, sheet_name='Data')
#             if col_widths is not None:
#                 worksheet = writer.sheets['Data']
#                 for idx, width in enumerate(col_widths, 1):
#                     column_letter = openpyxl.utils.get_column_letter(idx)
#                     worksheet.column_dimensions[column_letter].width = width
    
#     print(f"Создано {len(organizations)} файлов в папке '{output_folder}'")

if __name__ == "__main__":
    # Настройки
    # input_file = 'гепатиты 244-п февраль.xlsx'  # Укажите путь к вашему файлу
    # column_name = 'Прикрепление'  # Укажите название столбца с организациями
    # output_folder = '244'  # Название папки для результатов
    # sheet = 'Результат'#'asulon+'#'region+'
    # head_line=9
    # prefix='Гепатиты 244-п февраль'
    
    vipnets = {}
    with open('act_addr.txt') as in_file:
        csv_reader = csv.reader(in_file)
        for row in csv_reader:
            vipnets[row[0]]=row[1]
    
    # Запуск обработки
    # filter_and_save_organizations(
    #     input_file=input_file,
    #     column_name=column_name,
    #     output_folder=output_folder,
    #     sheet=sheet,
    #     head_line=head_line,
    #     prefix=prefix
    # )
    app = Application()
    app.mainloop()

In [ ]:
------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------
----------------------------ИСПОЛЬЗУЙ НИЖНИЙ БЛОКНОТ--------------------------------------------------------
------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------


In [5]:
import os
import csv
import threading
import openpyxl
from openpyxl.utils import get_column_letter
import pandas as pd
import tkinter as tk
from tkinter import ttk, filedialog, messagebox, simpledialog

class Application(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Фильтрация организаций")
        self.geometry("750x500")
        
        # self.style = ttk.Style()
        # self.style.theme_use('clam')
        
        # Переменные
        self.input_file = tk.StringVar()
        self.output_folder = tk.StringVar()
        self.column_name = tk.StringVar()
        self.sheet_name = tk.StringVar()
        self.header_row = tk.IntVar(value=1)
        self.status_text = tk.StringVar()
        self.progress_value = tk.DoubleVar()
        self.prefix = tk.StringVar()
        self.columns_list = []
        self.sheet_names = []
        self.preview_data = []

        # Флаг для остановки потока
        self.running = False
        
        self.create_widgets()
        self.protocol("WM_DELETE_WINDOW", self.on_close)
    
    def create_widgets(self):
        input_frame = ttk.LabelFrame(self, text="Параметры обработки")
        input_frame.pack(padx=10, pady=5, fill='x')
        
        # Выбор файла
        ttk.Label(input_frame, text="Входной файл:").grid(row=0, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.input_file, width=40).grid(row=0, column=1, padx=5, pady=2)
        ttk.Button(input_frame, text="Обзор...", command=self.browse_file).grid(row=0, column=2, padx=5, pady=2)
        
        # Выбор листа
        self.sheet_label = ttk.Label(input_frame, text="Лист в книге:")
        self.sheet_label.grid(row=1, column=0, sticky='w', padx=5, pady=2)
        self.sheet_combobox = ttk.Combobox(input_frame, textvariable=self.sheet_name, state="readonly", width=20)
        self.sheet_combobox.grid(row=1, column=1, sticky='w', padx=5, pady=2)
        
        # Выбор строки заголовка
        ttk.Label(input_frame, text="Строка с заголовками:").grid(row=2, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.header_row, width=5).grid(row=2, column=1, sticky='w', padx=5, pady=2)
        ttk.Button(input_frame, text="Предпросмотр", command=self.show_preview).grid(row=2, column=2, padx=5, pady=2)
        
        # Выбор столбца
        ttk.Label(input_frame, text="Столбец с организациями:").grid(row=3, column=0, sticky='w', padx=5, pady=2)
        self.column_combobox = ttk.Combobox(input_frame, textvariable=self.column_name, state="readonly", width=20)
        self.column_combobox.grid(row=3, column=1, sticky='w', padx=5, pady=2)
        
        # Выбор папки
        ttk.Label(input_frame, text="Папка для результатов:").grid(row=4, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.output_folder, width=40).grid(row=4, column=1, padx=5, pady=2)
        ttk.Button(input_frame, text="Обзор...", command=self.browse_output).grid(row=4, column=2, padx=5, pady=2)

        # prefix
        ttk.Label(input_frame, text="Префикс для названия файлов:").grid(row=5, column=0, sticky='w', padx=5, pady=2)
        ttk.Entry(input_frame, textvariable=self.prefix, width=40).grid(row=5, column=1, padx=5, pady=2)
        self.prefix.set('Гепатиты 244-п февраль')
        
        # Прогресс и статус
        progress_frame = ttk.Frame(self)
        progress_frame.pack(padx=10, pady=10, fill='x')
        self.progress = ttk.Progressbar(progress_frame, variable=self.progress_value, maximum=100)
        self.progress.pack(fill='x')
        
        status_frame = ttk.Frame(self)
        status_frame.pack(padx=10, pady=5, fill='x')
        ttk.Label(status_frame, textvariable=self.status_text).pack(side='left')
        
        # Кнопки
        button_frame = ttk.Frame(self)
        button_frame.pack(padx=10, pady=10)
        ttk.Button(button_frame, text="Старт", command=self.start_processing).pack(side='left', padx=5)
        ttk.Button(button_frame, text="Выход", command=self.on_close).pack(side='left', padx=5)
        
    def browse_file(self):
        filetypes = (("Excel/CSV файлы", "*.xlsx *.xls *.csv"), ("Все файлы", "*.*"))
        filename = filedialog.askopenfilename(filetypes=filetypes)
        if filename:
            self.input_file.set(filename)
            self.load_file_metadata(filename)
    
    def load_file_metadata(self, filename):
        try:
            if filename.lower().endswith(('.xls', '.xlsx')):
                self.load_sheets(filename)
                self.load_columns()
            else:
                self.load_columns()
            
            self.show_preview()
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Ошибка загрузки файла: {str(e)}")
    
    def load_sheets(self, filename):
        try:
            wb = openpyxl.load_workbook(filename, read_only=True)
            self.sheet_names = wb.sheetnames
            wb.close()
            self.sheet_combobox['values'] = self.sheet_names
            self.sheet_name.set(self.sheet_names[0])
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось прочитать листы: {str(e)}")
    
    def load_columns(self):
        try:
            header_row = self.header_row.get() - 1  # Конвертируем в 0-based индекс
            filename = self.input_file.get()
            
            if filename.endswith(('.xls', '.xlsx')):
                df = pd.read_excel(
                    filename,
                    sheet_name=self.sheet_name.get(),
                    header=header_row,
                    nrows=0
                )
            else:
                df = pd.read_csv(
                    filename,
                    header=header_row,
                    nrows=0,
                    encoding='utf-8'
                )
            
            self.columns_list = df.columns.tolist()
            self.column_combobox['values'] = self.columns_list
            if self.columns_list:
                self.column_name.set(self.columns_list[0])
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось прочитать заголовки: {str(e)}")
    
    def show_preview(self):
        try:
            filename = self.input_file.get()
            if not filename:
                return
            
            # Чтение первых 15 строк для предпросмотра
            nrows = 15
            header_row = self.header_row.get() - 1
            
            if filename.endswith(('.xls', '.xlsx')):
                df = pd.read_excel(
                    filename,
                    sheet_name=self.sheet_name.get(),
                    header=None,
                    nrows=nrows
                )
            else:
                df = pd.read_csv(
                    filename,
                    header=None,
                    nrows=nrows,
                    encoding='utf-8'
                )
            
            # Создание окна предпросмотра
            preview_window = tk.Toplevel(self)
            preview_window.title("Предпросмотр данных")
            
            # Создаем Treeview с прокруткой
            frame = ttk.Frame(preview_window)
            frame.pack(expand=True, fill='both', padx=5, pady=5)
            
            tree = ttk.Treeview(frame, columns=list(range(len(df.columns))), show="headings")
            vsb = ttk.Scrollbar(frame, orient="vertical", command=tree.yview)
            hsb = ttk.Scrollbar(frame, orient="horizontal", command=tree.xview)
            tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)
            
            tree.grid(row=0, column=0, sticky='nsew')
            vsb.grid(row=0, column=1, sticky='ns')
            hsb.grid(row=1, column=0, sticky='ew')
            frame.grid_columnconfigure(0, weight=1)
            frame.grid_rowconfigure(0, weight=1)
            
            # Настройка столбцов
            for i in range(len(df.columns)):
                tree.heading(i, text=f"Col {i+1}")
                tree.column(i, width=100, anchor='w')
            
            # Вставка данных
            for index, row in df.iterrows():
                tree.insert("", "end", values=tuple(row))
            
            # Подсветка строки с заголовками
            if header_row < len(df):
                tree.tag_configure('header', background='#b3e6ff')
                children = tree.get_children()
                if header_row < len(children):
                    tree.item(children[header_row], tags=('header',))
            
            # Обработка двойного клика
            def on_double_click(event):
                # Получаем выбранную строку и столбец
                item = tree.identify_row(event.y)
                column = tree.identify_column(event.x)
                
                if item and column:
                    # Получаем индекс строки
                    children = tree.get_children()
                    row_index = children.index(item) + 1  # +1 так как нумерация с 1
                    
                    # Получаем индекс столбца
                    col_index = int(column.replace("#", "")) - 1  # Преобразуем "#1" в 0
                    
                    # Устанавливаем новую строку заголовков
                    self.header_row.set(row_index)
                    self.load_columns()
                    
                    # Устанавливаем выбранный столбец
                    if self.columns_list and col_index < len(self.columns_list):
                        self.column_name.set(self.columns_list[col_index])
                    
                    # Закрываем окно предпросмотра
                    preview_window.destroy()
            
            # Привязываем обработчик двойного клика
            tree.bind("<Double-1>", on_double_click)
            
            # Кнопка закрытия
            button_frame = ttk.Frame(preview_window)
            button_frame.pack(fill='x', padx=5, pady=5)
            ttk.Button(button_frame, text="Закрыть", command=preview_window.destroy).pack(side='right')
            
            # Установка размера окна
            preview_window.geometry("800x400")
            
            # Инструкция для пользователя
            instruction = ttk.Label(
                preview_window,
                text="Дважды щелкните по ячейке, чтобы выбрать строку заголовков и столбец для фильтрации",
                font=('Arial', 10)
            )
            instruction.pack(pady=5)
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Ошибка предпросмотра: {str(e)}")
    
    def browse_output(self):
        folder = filedialog.askdirectory()
        if folder:
            self.output_folder.set(folder)
    
    def start_processing(self):
        if not self.validate_input():
            return
        # Запуск обработки в отдельном потоке
        self.running = True
        self.processing_thread = threading.Thread(target=self.process_data)
        self.processing_thread.start()
    
    def validate_input(self):
        if not self.input_file.get():
            messagebox.showwarning("Ошибка", "Выберите входной файл!")
            return False
        if self.input_file.get().endswith(('.xls', '.xlsx')) and not self.sheet_name.get():
            messagebox.showwarning("Ошибка", "Выберите лист в книге!")
            return False
        if not self.column_name.get():
            messagebox.showwarning("Ошибка", "Выберите столбец с организациями!")
            return False
        return True
    
    def process_data(self):
        vipnets = {}
        with open('act_addr.txt') as in_file:
            csv_reader = csv.reader(in_file)
            for row in csv_reader:
                vipnets[row[0]]=row[1]
        try:
            self.after(0, self.update_status, "Начало обработки...")
            
            input_file = self.input_file.get()
            column_name = self.column_name.get()
            output_folder = self.output_folder.get() or 'output'
            sheet = self.sheet_name.get() if self.sheet_name.get() else None
            header_row = self.header_row.get() - 1
            
            # Загрузка данных
            if input_file.endswith('.csv'):
                df = pd.read_csv(input_file, header=header_row)
                col_widths = None
            else:
                wb = openpyxl.load_workbook(input_file)
                ws = wb[sheet] if sheet else wb.active
                
                # Чтение ширины столбцов
                col_widths = []
                for col in range(1, ws.max_column + 1):
                    col_letter = get_column_letter(col)
                    col_widths.append(
                        ws.column_dimensions[col_letter].width 
                        if col_letter in ws.column_dimensions 
                        else 8.43
                    )
                
                df = pd.read_excel(input_file, sheet_name=sheet, header=header_row)
                wb.close()

            # Обработка данных
            organizations = df[column_name].unique()
            total = len(organizations)
            os.makedirs(output_folder, exist_ok=True)
            
            for i, org in enumerate(organizations, 1):
                if not self.running:  # Проверка флага
                    break
                org_data = df[df[column_name] == org]
                clean_org_name = "".join([c if c.isalnum() or c in (' ', '_') else '' for c in str(org)])
                if org in vipnets:
                    output_path = os.path.join(output_folder, self.prefix.get()+' '+clean_org_name+vipnets[org]+'.xlsx')
                else:
                    print('нет правила для организации:', org)
                    output_path = os.path.join(output_folder, f"{clean_org_name}.xlsx")
                
                with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
                    org_data.to_excel(writer, index=False, sheet_name='Data')
                    if col_widths is not None:
                        worksheet = writer.sheets['Data']
                        for idx, width in enumerate(col_widths, 1):
                            worksheet.column_dimensions[get_column_letter(idx)].width = width
                progress = (i / total) * 100
                self.after(0, self.update_progress, progress)
                self.after(0, self.update_status, f"Обработано: {i} из {total} ({org})")
            
            if self.running:
                self.after(0, self.update_status, "Обработка завершена успешно!")
                self.after(0, lambda: messagebox.showinfo(
                    "Готово", 
                    f"Создано {total} файлов в папке:\n{output_folder}"
                ))
        
        except Exception as e:
            self.after(0, lambda: messagebox.showerror("Ошибка", str(e)))
            self.after(0, self.update_status, "Ошибка обработки!")
        finally:
            self.after(0, self.reset_progress)
            self.running = False
    
    def update_progress(self, value):
        self.progress_value.set(value)
    
    def reset_progress(self):
        self.progress_value.set(0)
    
    def update_status(self, message):
        self.status_text.set(message)
    
    def on_close(self):
        if messagebox.askokcancel("Выход", "Вы уверены, что хотите выйти?"):
            self.running = False  # Остановка фонового потока
            if hasattr(self, 'processing_thread') and self.processing_thread.is_alive():
                self.processing_thread.join(timeout=1)  # Ожидание завершения потока
            self.destroy()

if __name__ == "__main__":
    app = Application()
    app.mainloop()

нет правила для организации: ГАУЗ СО ГИБ г. Н.Тагил


#------------------------------------>>>4. PCR COVID-19<<<-------------------------------------
if not pcr_covid_ready:
    pcr_covid_file="S:\\14 - Лаборатория\\списки обследованных на cov19\\обследованные на cov19 2024.ods"
    pcr_covid=pd.read_excel(pcr_covid_file, sheet_name='Данные')
    pcr_covid.dropna()
    pcr_covid_selected = pcr_covid.loc[(pcr_covid['Название организации-заказчика'] != 'ГАУЗ СО ГИБ г.Н.Тагил')&(pcr_covid['Дата отчетности'].dt.month == month)]
    pcr_covid_ready = True
    
for index, row in pcr_covid_selected.iterrows():
    zap_count+=1
    usl_count+=1
    date=row['Дата отчетности'].date().isoformat()
    npolis=('%f'%row['ОМС']).rstrip('0').rstrip('.')
    code_npr_mo,code_npr_doctor='660010', '38945'
    if row['Название организации-заказчика']=='ГАУЗ СО ГБ №4':
        code_npr_mo,code_npr_doctor='660013', '7654'
    history_num=(date+npolis)[:24]
    if type(row['Отчество']) == str:
        patronymic=row['Отчество'].strip()
    else:
        patronymic=row['Отчество'] = ''
    zap, per = generate_zap_per_app(zap_count, 
                                    usl_count, 
                                    history_num=history_num,
                                    date=date,
                                    npr_mo=code_npr_mo,
                                    npr_doctor=code_npr_doctor,
                                    podr='90000', 
                                    profil='34',
                                    prvs='26',
                                    iddokt='7229',
                                    ds='J06.9',
                                    code_usl='A26.08.008.102', 
                                    npolis=npolis,
                                    fam=row['Фамилия'].strip(),
                                    im=row['Имя'].strip(),
                                    ot=patronymic,
                                    w=sex_is_magic(patronymic),
                                    dr=row['Дата рождения'].date().isoformat(),
                                    snils=parse_snils(row['СНИЛС']),
                                    vpolis=3, 
                                    smo="66004", 
                                    p_cel=12,
                                    razdel_usl=301,
                                    kol_usl=1,
                                    usl_ok=3,
                                    vidpom=13,
                                    npr_num=history_num,
                                    npr_type=4,
                                    idsp=28,
                                    rslt=314,
                                    for_pom=3,
                                    okato="65476000000",
                                    mes=1101
                                   )
    pers.append(per)
    zaps.append(zap)
    print("added pcr-grip:", zap, per)
print(datetime.now())